In [1]:
from discussion_agents.cog.agent.react import ReActAgent
from langchain_community.chat_models.openai import ChatOpenAI
from discussion_agents.cog.prompts.react import REACT_INSTRUCTION_FEVER, FEVER_FEWSHOT_EXAMPLES

import dotenv
import os

dotenv.load_dotenv(".env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=openai_api_key)

In [2]:
agent = ReActAgent(llm=llm)

In [4]:
question = 'Brad Wilk died before being a drummer for Greta.'
agent.generate(question=question, examples=FEVER_FEWSHOT_EXAMPLES, prompt=REACT_INSTRUCTION_FEVER)

<===================================================================>
Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION.  and Action can be two types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
Claim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
Thought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.
Action 1: Search[Nikolaj Coster-Waldau]
Observation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime La

[('Thought: I need to search for Brad Wilk to find out if he died before being a drummer for Greta.',
  'Action: Search[Brad Wilk]',
  "Observation 1: Could not find [Brad Wilk]. Similar: ['Brad Wilk', 'Rage Against the Machine', 'Audioslave', 'Wilk', 'Prophets of Rage', 'Tim Commerford', 'Zack de la Rocha', 'Tom Morello', 'Greta (band)', 'Show Me How to Live (song)']"),
 ("Thought: Since we couldn't find information on Brad Wilk, I should search for Greta (band) to see if Brad Wilk was a drummer for them.",
  'Action: Search[Greta (band)]',
  'Observation 2: Greta was an American hard rock band formed in 1990 by Paul Plagens, Kyle Baer, Josh Gordon, and Brad Wilk.Wilk left the band in 1991 to join Rage Against the Machine, and was replaced on the drums by Scott Carneghi who went on to co-found the band Buffalocomotive in 2012.The band signed a two-record deal with Mercury Records in 1993 and released their debut album, No Biting, on September 21 of that year. Their second and final al